In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from sqlalchemy import create_engine
import sqlite3

# Creating a dictionary

In [4]:
from os import walk, path
dic_csv = {}
df_var = {} 
for root,d_names,f_names in walk("./data/"):
    for f in f_names:
        filename = f.rsplit('.', 1)[0]
        dic_csv[path.join(root, f)] = ('processed_' + filename) if ('processed' in root) else filename

    

In [5]:
def make_df(dic):
    for key in dic:
        df_var[dic[key]] = pd.read_csv(key, delimiter=";")
make_df(dic_csv)

/home/ricardo/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3337: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
def separate(df, column):
    list = df[column].unique().tolist()
    dic = {}
    for l in list:
        dic[l] = df[df[column]==l]
    return dic

In [7]:
def concat_train_test(dic):
    lis = [d for d in list(dic.values()) if ('test' in d or 'train' in d)]
    train = []
    test = []
    names = []
    for l in lis:
        names.append(l.split("_")[0])
        if (l.split("_")[1] == 'train'):
            train.append(l)
        else:
            test.append(l)
    names = list(set(names))
    
    for n in names:
        t1 = [d for d in train if n in d][0]
        t2 = [d for d in test if n in d][0]
        
        df_var[n] = pd.concat([df_var[t1],df_var[t2]])
concat_train_test(dic_csv)

# Auxiliary functions to process data

In [8]:
from datetime import date

def getDate(d):
    year = 1900+int(str(d)[0:2])
    month = int(str(d)[2:4])
    gender = 'M' if (month < 50) else 'F'
    month = month if (month < 50) else month - 50
    day = int(str(d)[4:6])
    return {'year': year, 'month': month, 'day': day, 'gender': gender}

def getGender(df):
    list = []
    for row in df.itertuples(index = True):
        d = getDate(row.birth_number)
        birth_number = row.birth_number 
        if (d['gender'] == 'F'):
            birth_number -= 5000
        list.append([row.client_id, birth_number, d['gender'], row.district_id])
    
    return pd.DataFrame(list, columns=['client_id', 'birth_number', 'gender', 'district_id'])

print(getGender(df_var['client']))


      client_id  birth_number gender  district_id
0             1        701213      F           18
1             2        450204      M            1
2             3        401009      F            1
3             4        561201      M            5
4             5        600703      F            5
...         ...           ...    ...          ...
5364      13955        451030      F            1
5365      13956        430406      M            1
5366      13968        680413      M           61
5367      13971        621019      F           67
5368      13998        530812      F           74

[5369 rows x 4 columns]


In [ ]:
def date_range(df,min_year, max_year):
    
    

In [9]:
def getColumns(df):
    sum = 0
    for key in df:
        if not 'train' in key and not 'test' in key:
            print('{} -> {}'.format(key, len(df_var[key].columns)))
            print('{}\n'.format(df_var[key].columns))

            sum = sum + len(df_var[key].columns)
    print(sum)

In [238]:
#credit in cash', 'collection from another bank', nan,'withdrawal in cash', 'remittance to another bank',
#       'credit card withdrawal'

def n_credit_cash(col):
    return sum(col == 'credit in cash')

def n_collection_from_another_bank(col):
    return sum(col == 'collection from another bank')

def n_withdrawal_in_cash(col):
    return sum(col == 'withdrawal in cash')

def n_remittance_to_another_bank(col):
    return sum(col == 'remittance to another bank')

def n_credit_card_withdrawal(col):
    return sum(col == 'credit card withdrawal')

def n_interest_credited(col):
    return sum(col == 'none')



def n_credit(col):
    return sum(col == 'credit')

def n_withdrawal(col):
    return sum(col == 'withdrawal') + sum(col == 'withdrawal in cash')

# Merging information

In [ ]:
# Copying information from dictonary into dataframes

client = df_var['client'].copy()
disp = df_var['disp'].copy()
card = df_var['card'].copy()
loan = df_var['loan'].copy()
account = df_var['account'].copy()
card = card.rename({'type': 'card_type'}, axis=1)
disp = disp.rename({'type': 'disp_type'}, axis=1)

# Merging informations

merge1 = pd.merge(client, disp[disp['disp_type'] =='OWNER'], on ='client_id')
merge2 = pd.merge(merge1, account, on = 'account_id')
merge3 = pd.merge(merge2, card[['card_id', 'disp_id', 'card_type']], on = 'disp_id', how = 'left')
merge4 = pd.merge(loan, merge3, on = 'account_id')

# Dropping some columns
merge4.drop(['client_id', 'disp_id', 'disp_type', 'card_id'], axis = 1, inplace = True)

# Renaming columns
merge4.rename({
    'date_x': 'loan_date', 
    'district_id_x': 'client_district', 
    'district_id_y': 'account_district',
    'date_y': 'account_date'
}, axis = 1, inplace = True)

In [167]:
# trans_processed = convertToDate(trans_df,'trans_train','date', False)[['account_id', 'date', 'balance']]
#    trans_processed.sort_values(by = ['account_id', 'date'])
#    t = trans_processed.groupby('account_id').tail(1)

# Processing transactions information 

trans = df_var['trans'].copy()
trans = trans.rename({
    'type': 'trans_type',
    'date': 'trans_date', 
    'amount': 'trans_amount',
    'balance': 'trans_balance'}, axis=1)
trans.sort_values(by = ['account_id', 'trans_date'])
trans.drop(['trans_id', 'k_symbol', 'account', 'bank'], axis=1, inplace = True)

#t = trans.groupby('account_id').tail(1)

# Merging trans with previous information

loan_info = pd.merge(merge4, trans, on = 'account_id')
loan_info.fillna('none', inplace = True)

In [265]:
#Columns that are not going to be changed
cols = loan_info.columns.difference(['trans_date', 'trans_amount', 'trans_balance', 'trans_type', 'operation'], sort = False).tolist()

# Group by operation

l = loan_info.groupby(by = cols, as_index = False).agg({
                    'trans_date': ['min', 'max'],
                    'trans_amount': ['min', 'max', 'mean', 'std', 'last'],
                    'trans_balance': ['min', 'max', 'mean', 'std', 'last'],
                    'trans_type':[n_credit, n_withdrawal],
                    'operation':[n_credit_cash, n_collection_from_another_bank, 
                                 n_withdrawal_in_cash, n_remittance_to_another_bank,
                                 n_credit_card_withdrawal, n_interest_credited]
                })

#Change columns name 
l.columns = ['%s%s' % (a, '_%s' % b if b else '') for a, b in df.columns]l
#l.sort_values(by = ['loan_id'])

loan_id account_id loan_date  amount duration payments status  \
                                                                    
0      4959          2    940105   80952       24     3373      1   
1      4961         19    960429   30276       12     2523     -1   
2      4962         25    971208   30276       12     2523   none   
3      4967         37    981014  318480       60     5308   none   
4      4968         38    980419  110736       48     2307   none   
..      ...        ...       ...     ...      ...      ...    ...   
677    7294      11327    980927   39168       24     1632   none   
678    7295      11328    980718  280440       60     4674   none   
679    7304      11349    951029  419880       60     6998      1   
680    7305      11359    960806   54024       12     4502      1   
681    7308      11362    961227  129408       24     5392      1   

    birth_number client_district account_district  ... trans_balance           \
                                                   ...           std     last   
0         450204               1                1  ...  12061.705682  27855.2   
1         395423              21               21  ...  15039.248405  15854.0   
2         620209              68               68  ...  22476.347980  31621.9   
3         520826              20               20  ...  23451.647180  16367.8   
4         405130              19               19  ...  10950.811533  37416.7   
..           ...             ...              ...  ...           ...      ...   
677       795222               7                7  ...  14622.737828  62119.2   
678       525909              54               54  ...  17484.955759  31303.2   
679       456030               1                1  ...  27879.396857  24704.4   
680       680413              61               61  ...  15469.988113  25697.2   
681       626019              67               67  ...  11709.142375  24199.5   

    trans_type                  operation                                 \
      n_credit n_withdrawal n_credit_cash n_collection_from_another_bank   
0           22           32             2                             10   
1           46           34             9                             13   
2           51          113            36                              0   
3           32           84            18                              0   
4           23           32             6                              9   
..         ...          ...           ...                            ...   
677         23           31            13                              0   
678         47           59            28                              0   
679         11            7             2                              5   
680         45          102             1                             22   
681         29           78            16                              0   

                                                       \
    n_withdrawal_in_cash n_remittance_to_another_bank   
0                     27                            5   
1                     34                            0   
2                     70                           43   
3                     48                           36   
4                     19                           13   
..                   ...                          ...   
677                   25                            6   
678                   59                            0   
679                    6                            1   
680                   66                           34   
681                   38                           40   

                                                  
    n_credit_card_withdrawal n_interest_credited  
0                          0                  10  
1                          0                  24  
2                          0                  15  
3                          0                  14  
4              

In [263]:
trans[trans['account_id'] == 2]

,account_id,trans_date,trans_type,operation,trans_amount,trans_balance
44,2,930226,credit,credit in cash,1100.0,1100.0
84,2,930312,credit,collection from another bank,20236.0,21336.0
129,2,930328,credit,credit in cash,3700.0,25036.0
168,2,930331,credit,NaN,13.5,25049.5
204,2,930412,credit,collection from another bank,20236.0,45285.5
261,2,930427,withdrawal,withdrawal in cash,11000.0,34285.5
326,2,930430,credit,NaN,109.5,34394.9
389,2,930512,credit,collection from another bank,20236.0,54630.9
465,2,930527,withdrawal,withdrawal in cash,17600.0,37030.9
552,2,930531,credit,NaN,144.7,37175.6


In [201]:
card[card['disp_id']==2166]

,card_id,disp_id,card_type,issued


In [204]:
merge4[merge4['card_type']=='classic']

,loan_id,account_id,date_x,amount,duration,payments,status,client_id,birth_number,district_id_x,disp_id,disp_type,district_id_y,frequency,date_y,card_id,card_type
40,6577,7753,940311,51696,24,2154,1.0,9593,685128,74,9285,OWNER,74,monthly issuance,930208,1005.0,classic
106,5595,3050,940902,82896,12,6908,1.0,3687,720206,49,3687,OWNER,60,monthly issuance,930508,577.0,classic
245,7243,11042,951204,217152,36,6032,1.0,13576,740707,72,13268,OWNER,72,monthly issuance,950129,1225.0,classic
295,6550,7614,960518,320592,48,6679,1.0,9427,706109,62,9119,OWNER,62,monthly issuance,950208,989.0,classic
306,5311,1766,960615,30060,60,501,1.0,2141,510906,28,2141,OWNER,28,monthly issuance,940903,337.0,classic
321,6992,9790,960727,133968,48,2791,1.0,12050,430530,52,11742,OWNER,52,monthly issuance,951109,1134.0,classic
333,7305,11359,960806,54024,12,4502,1.0,13968,680413,61,13660,OWNER,61,monthly issuance,941001,1247.0,classic
337,5088,544,960822,91152,24,3798,1.0,655,421210,32,655,OWNER,32,monthly issuance,950515,119.0,classic
438,6938,9504,970310,341280,60,5688,NaN,11701,490414,9,11393,OWNER,9,monthly issuance,960129,1118.0,classic
488,7119,10361,970602,86880,12,7240,NaN,12750,405525,21,12442,OWNER,21,monthly issuance,960621,1173.0,classic


In [207]:
card[card['card_id']==17]

,card_id,disp_id,card_type,issued
21,17,127,classic,980607


<>:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<>:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
<ipython-input-158-8140e5144396>:1: SyntaxWarning: 'int' object is not subscriptable; perhaps you missed a comma?
  530812[1]


TypeError: 'int' object is not subscriptable

In [ ]:
def db():
    CSV_files = {'account': account, 'card': card_train, 'client': client, 'disp': disp, 'disctrict': district, 'loan': loan_train, 'trans': trans_train}
    conn = sqlite3.connect('database.db')
    # Correr só uma vez
    if not os.path.exists('./database.db'):
        for f in CSV_files:
            CSV_files[f].to_sql(f, conn)
    query = """
        SELECT loan_id, loan.account_id, status
        FROM loan, account
        WHERE loan.account_id == account.account_id
        """
         
    pd.read_sql_query(query, conn)
    
def getCorr(df, size=(11,9)):
    corr = df.corr()
    ax = plt.subplots(figsize=size)
    ax = sns.heatmap(corr,  annot = True, linewidths=.1, mask = np.triu(corr), cmap='coolwarm')

    def checkUniqueCategoricalVars():
    for f in CSV_files:
        print("{}:\n {}\n\n".format(f, CSV_files[f].select_dtypes(include=['object']).nunique()))